## Imports

In [19]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras import layers, models
import pathlib
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## Read Images

In [20]:
base_dir = "dataset/"
base_dir = pathlib.Path(base_dir)
images = list(base_dir.glob('*/*.jpg'))
print(type(images))

<class 'list'>


## Split data

In [21]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  base_dir,
  validation_split=0.2,
  subset="training",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  base_dir,
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 42 files belonging to 2 classes.
Using 34 files for training.
Found 42 files belonging to 2 classes.
Using 8 files for validation.


In [28]:
train_ds

<BatchDataset shapes: ((None, 180, 180, 3), (None,)), types: (tf.float32, tf.int32)>

## Model creation

In [35]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [36]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
2/2 [==============================] - 0s 127ms/step - loss: 0.7369 - accuracy: 0.6176 - val_loss: 0.5762 - val_accuracy: 0.6250
Epoch 2/3
2/2 [==============================] - 0s 66ms/step - loss: 0.7347 - accuracy: 0.5000 - val_loss: 0.5577 - val_accuracy: 0.6250
Epoch 3/3
2/2 [==============================] - 0s 61ms/step - loss: 0.7137 - accuracy: 0.5000 - val_loss: 0.4561 - val_accuracy: 1.0000


In [47]:
loss,acc = model.evaluate(val_ds)

1/1 [==============================] - 0s 0s/step - loss: 0.4561 - accuracy: 1.0000


In [48]:
y_pred = model.predict_classes(val_ds)

In [53]:
model.save('model/lemon_melon.h5') 

In [54]:
new_model = tf.keras.models.load_model('model/lemon_melon.h5')

In [55]:
loss,acc = new_model.evaluate(val_ds)

1/1 [==============================] - 0s 0s/step - loss: 0.4561 - accuracy: 0.6250


In [52]:
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_2 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 87, 87, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 41, 41, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 20, 20, 32)       